# Get log probs

Certain chat models can be configured to return token-level log probabilities. This guide walks through how to get logprobs for a number of models.

In [1]:
%pip install -qU langchain-openai

Note: you may need to restart the kernel to use updated packages.


In [1]:
from langchain_openai import ChatOpenAI

# For the OpenAI API to return log probabilities we need to configure the `logprobs` param
llm = ChatOpenAI(model="gpt-3.5-turbo-0125").bind(logprobs=True)

msg = llm.invoke(("human", "how are you today"))

In [2]:
msg.additional_kwargs["logprobs"]["content"][:5]

[{'token': 'I', 'bytes': [73], 'logprob': -0.21393852, 'top_logprobs': []},
 {'token': "'m",
  'bytes': [39, 109],
  'logprob': -0.34835616,
  'top_logprobs': []},
 {'token': ' just',
  'bytes': [32, 106, 117, 115, 116],
  'logprob': -0.19935185,
  'top_logprobs': []},
 {'token': ' a',
  'bytes': [32, 97],
  'logprob': -0.001974669,
  'top_logprobs': []},
 {'token': ' computer',
  'bytes': [32, 99, 111, 109, 112, 117, 116, 101, 114],
  'logprob': -0.05223269,
  'top_logprobs': []}]

In [3]:
ct = 0
full = None
for chunk in llm.stream(("human", "how are you today")):
    if ct < 5:
        full = chunk if full is None else full + chunk
        print(full.additional_kwargs["logprobs"]["content"])
    else:
        break
    ct += 1

[]
[{'token': 'I', 'bytes': [73], 'logprob': -0.21393852, 'top_logprobs': []}]
[{'token': 'I', 'bytes': [73], 'logprob': -0.21393852, 'top_logprobs': []}, {'token': "'m", 'bytes': [39, 109], 'logprob': -0.34287196, 'top_logprobs': []}]
[{'token': 'I', 'bytes': [73], 'logprob': -0.21393852, 'top_logprobs': []}, {'token': "'m", 'bytes': [39, 109], 'logprob': -0.34287196, 'top_logprobs': []}, {'token': ' just', 'bytes': [32, 106, 117, 115, 116], 'logprob': -0.18108529, 'top_logprobs': []}]
[{'token': 'I', 'bytes': [73], 'logprob': -0.21393852, 'top_logprobs': []}, {'token': "'m", 'bytes': [39, 109], 'logprob': -0.34287196, 'top_logprobs': []}, {'token': ' just', 'bytes': [32, 106, 117, 115, 116], 'logprob': -0.18108529, 'top_logprobs': []}, {'token': ' a', 'bytes': [32, 97], 'logprob': -0.001870452, 'top_logprobs': []}]


In [5]:
def multiply(a: int, b: int) -> int:
    """mulitply"""
    return a * b


full = None
llm_with_tools = ChatOpenAI(model="gpt-3.5-turbo-0125").bind_tools([multiply])
i = 0
for chunk in llm_with_tools.stream("what's 3 * 5"):
    print(f"{i}: before")
    print(full.additional_kwargs if full else None)
    if full is None:
        full = chunk
    else:
        print(f"{i}: during")
        print(full.additional_kwargs, chunk.additional_kwargs)

        full += chunk.copy()
        print(f"{i}: after")
        print(full.additional_kwargs)

    # full = chunk if full is None else full + chunk
    # print(chunk)
    # print(full)
    i += 1

0: before
None
1: before
{'tool_calls': [{'index': 0, 'id': 'call_dlDKDSkuHREHH3KHJI5gYrRr', 'function': {'arguments': '', 'name': 'multiply'}, 'type': 'function'}]}
1: during
{'tool_calls': [{'index': 0, 'id': 'call_dlDKDSkuHREHH3KHJI5gYrRr', 'function': {'arguments': '', 'name': 'multiply'}, 'type': 'function'}]} {'tool_calls': [{'index': 0, 'id': None, 'function': {'arguments': '{"', 'name': None}, 'type': None}]}
1: after
{'tool_calls': [{'index': 0, 'id': 'call_dlDKDSkuHREHH3KHJI5gYrRr', 'function': {'arguments': '{"', 'name': 'multiply'}, 'type': 'function'}]}
2: before
{'tool_calls': [{'index': 0, 'id': 'call_dlDKDSkuHREHH3KHJI5gYrRr', 'function': {'arguments': '{"{"', 'name': 'multiply'}, 'type': 'function'}]}
2: during
{'tool_calls': [{'index': 0, 'id': 'call_dlDKDSkuHREHH3KHJI5gYrRr', 'function': {'arguments': '{"{"', 'name': 'multiply'}, 'type': 'function'}]} {'tool_calls': [{'index': 0, 'id': None, 'function': {'arguments': 'a', 'name': None}, 'type': None}]}
2: after
{'too

In [7]:
first_chunk = None
full = None
for chunk in (
    ChatOpenAI(model="gpt-3.5-turbo-0125").bind_tools([multiply]).stream("what's 3 * 5")
):
    first_chunk = chunk if first_chunk is None else first_chunk
    print(chunk)
    full = chunk if full is None else full + chunk

content='' additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_WHQtpoFThe9DNqknMLLzf4mb', 'function': {'arguments': '', 'name': 'multiply'}, 'type': 'function'}]}
content='' additional_kwargs={'tool_calls': [{'index': 0, 'id': None, 'function': {'arguments': '{"', 'name': None}, 'type': None}]}
content='' additional_kwargs={'tool_calls': [{'index': 0, 'id': None, 'function': {'arguments': 'a', 'name': None}, 'type': None}]}
content='' additional_kwargs={'tool_calls': [{'index': 0, 'id': None, 'function': {'arguments': '":', 'name': None}, 'type': None}]}
content='' additional_kwargs={'tool_calls': [{'index': 0, 'id': None, 'function': {'arguments': '3', 'name': None}, 'type': None}]}
content='' additional_kwargs={'tool_calls': [{'index': 0, 'id': None, 'function': {'arguments': ',"', 'name': None}, 'type': None}]}
content='' additional_kwargs={'tool_calls': [{'index': 0, 'id': None, 'function': {'arguments': 'b', 'name': None}, 'type': None}]}
content='' additional_kwargs={'tool

In [10]:
first_chunk

AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_WHQtpoFThe9DNqknMLLzf4mb', 'function': {'arguments': '{"{"aa":":33,","bb":":55}}', 'name': 'multiply'}, 'type': 'function'}]})

In [18]:
first_chunk.additional_kwargs

{'tool_calls': [{'index': 0,
   'id': 'call_WHQtpoFThe9DNqknMLLzf4mb',
   'function': {'arguments': '{"{"aa":":33,","bb":":55}}', 'name': 'multiply'},
   'type': 'function'}]}

In [16]:
full.additional_kwargs

{'tool_calls': [{'index': 0,
   'id': 'call_WHQtpoFThe9DNqknMLLzf4mb',
   'function': {'arguments': '{"{"aa":":33,","bb":":55}}', 'name': 'multiply'},
   'type': 'function'}],
 'finish_reason': 'tool_calls'}